Goals of this analysis: 
- Create **rolling mean** of data to smooth out some variation
- Make separate **linear regressions** for distinct treatment periods

In [2]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns

In [6]:
data=pd.read_excel('PerformanceDataFake.xlsx')
data.head(n=15)

,Subject,Measure1,Measure2,Measure3,Measure4,Measure5,TestPhase,SessionLabel2,SessionLabel,LastTreatmentSession,In-TreatmentSessionNumber
0,1,95,14,3,93,39,-1,P1,P6,0,NaN
1,1,28,15,51,54,28,-1,P2,P5,0,NaN
2,1,70,77,13,35,63,-1,P3,P4,0,NaN
3,1,5,65,45,62,50,-1,P4,P3,0,NaN
4,1,38,62,76,53,63,-1,P5,P2,0,NaN
5,1,25,85,29,4,75,-1,P6,P1,0,NaN
6,1,56,23,20,64,43,0,F1,F4,0,1.0
7,1,84,7,49,26,64,0,F2,F3,0,2.0
8,1,33,56,37,51,81,0,F3,F2,0,3.0
9,1,70,59,33,27,72,0,F4,F1,1,4.0


Each row is one trial for one subject. Several measures are taken, and metadata is included to denote whether this trial occured before treatment began, during treatment, or after treatment ended.

In [13]:
#Sanitize the inputs a little
df = data

#for this analysis, we don't need post-treatment -- remove it
df = df[df['SessionLabel2'].str.contains('R1')==False] 

#For our linear regression, we need NaNs replaced with 0
df['In-TreatmentSessionNumber'].fillna(0,inplace=True)

#And change TestPhase from -1,0 to 0,1
df = df.assign(TestPhase = [0 if TestPhase == -1 else 1 for TestPhase in df['TestPhase']]) #Convert the -1 to 0

#Variables to loop through
subjects=df['Subject'].unique()
measures=['Measure1','Measure2','Measure3','Measure4','Measure5']

C:\Users\andre\AppData\Local\Temp\ipykernel_23992\1716536942.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['In-TreatmentSessionNumber'].fillna(0,inplace=True)


The above warning is a false positive **(** and can be disabled with pd.set_option('mode.chained_assignment',None) **)**

We need to make two functions now -- one to implement a rolling mean, and one to plot our data and fit lines

In [14]:
#A little setup to allow us to do nested assignment
from collections import defaultdict
model = defaultdict(dict)
res = defaultdict(dict)